In [2]:
import pandas as pd
from coleta_dados import DadosFundamentalistas, VerificadorTicks
from chat_bots import chat_bot_fundamentalistas


In [3]:
from typing import Optional, List
from langchain_community.document_loaders import DataFrameLoader
from coleta_dados import DadosFundamentalistas
def tratando_dados_fundamentalistas(tic: str,
        data_inicio: Optional[str] = None,
        data_fim: Optional[str] = None,
        colunas_drop:Optional[List[str],None] = None
        ) -> List:
    if colunas_drop is None:
       colunas_drop = ['acoes_ordinarias', 'acoes_preferenciais', 'total',  'capex_doze_meses', 'fluxo_caixa_livre_doze_meses', 'retorno_sobre_capital_tangivel_inicial',
                    'retorno_sobre_capital_tangivel_inicial_pre_impostos','retorno_sobre_capital_investido_inicial_pre_impostos', 'preço_ncav', 'margem_ebitda',
                    'divida_liquida_ebitda', 'ev_ebitda', 'ev_receita_líquida', 'ev_fco','ev_fcf', 'ev_ativo_total','preço_vpa','preço_capital_giro', 'preço_fcf',
                    'caixa_equivalentes_caixa'] 
    
    dados_fundamentalistas = DadosFundamentalistas(tic=tic,data_inicio=data_inicio, data_fim=data_fim)
    dados = dados_fundamentalistas.dados_fundamentalistas_completo()
    dados.drop(columns=colunas_drop, inplace=True)
    dados['datas'] = dados['datas'].astype(str)
    loader = DataFrameLoader(dados,page_content_column='datas')
    dados_load = loader.load()
    return dados_load
    
    

In [ ]:
from langchain import PromptTemplate
def chat_bot_fundamentalistas(
    query: str, dados:list,api_secret: str = api_secret_groq
) -> str:
    prompt = PromptTemplate(
      input_variables = ['query', 'dados'],
      template = '''
                                                  You are a fundamental analyst, responsible for assessing the financial health, operational performance, and investment prospects of companies based on financial indicators. Your goal is to produce detailed, well-founded, and actionable analyses for investors.

                                                    The indicators you receive include the following data:

                                                    Resultados financeiros: receita_liquida, resultado_bruto, ebit, ebitda, lucro_liquido, lucro_por_acao.
                                                    Fluxos de caixa: fluxo_caixa_operacional, fluxo_caixa_investimentos, fluxo_caixa_financiamento, fluxo_caixa_livre_tres_meses.
                                                    Investimentos e despesas: capex_tres_meses.
                                                    Valuation: preço_lucro, preço_receita_líquida,  preço_fcf, preço_ebit, ev_ebit, ev_ebitda, enterprise_value, market_cap_empresa.
                                                    Dividendo: dividend_yield.
                                                    Balanço patrimonial: ativo_total, divida_bruta, divida_liquida, patrimonio_liquido, valor_patrimonial_acao.
                                                    Indicadores de rentabilidade e eficiência: retorno_sobre_capital_tangivel_inicial, retorno_sobre_capital_investido_inicial, retorno_sobre_patrimonio_liquido_inicial, margem_bruta, margem_liquida, giro_do_ativo_inicial, alavancagem_financeira.
                                                    Riscos e alavancagem:  passivo_patrimonio_liquido.
                                                    Your Task:
                                                    Analyze indicators: Identify the company’s strengths and weaknesses based on the provided indicators.
                                                    Generate actionable insights: Provide recommendations for investors, such as "buy," "hold," or "sell," with clear justifications based on the data.
                                                    Identify trends: Evaluate historical performance trends and possible implications for the future.
                                                    Communicate results: Present your analysis clearly and concisely, suitable for both beginners and experienced investors.
                                                    Example of Expected Response:

                                                    Evaluated Company: [company name or ticker]
                                                    Analysis Summary: Provide an overall assessment of the company based on the indicators.
                                                    Strengths: Highlight positive metrics, such as high net_income, strong margins, or low net_debt.
                                                    Weaknesses: Identify areas of concern, such as low net_margin or high net_debt_to_ebitda.
                                                    Recommendation: Based on your analysis, provide a recommendation (buy, hold, or sell) with justification.
                                                    
                                                                                                   
        REQUESTED ANALYSIS:
        Question: {query}
        Financial Data: {dados}
        
        
        Always response the questions in Portuguese.

      ''')

    model = ChatGroq(
            api_key=api_secret,
            #model="llama-3.3-70b-versatile",
            model = "llama-3.2-11b-vision-preview",
            temperature=0.5,
            stop_sequences=None,
        )

    llm_chain = prompt | model | StrOutputParser()

    response = llm_chain.invoke(input={'query':query, 'dados': dados})
    
    return response

In [12]:
dados = dados_fundamentalistas.dados_fundamentalistas_completo()

In [13]:
colunsa_drop = ['acoes_ordinarias', 
 'acoes_preferenciais', 
 'total', 
 'capex_doze_meses', 
 'fluxo_caixa_livre_doze_meses',
 'retorno_sobre_capital_tangivel_inicial',
 'retorno_sobre_capital_tangivel_inicial_pre_impostos',
'retorno_sobre_capital_investido_inicial_pre_impostos',
'preço_ncav',
'margem_ebitda',
'divida_liquida_ebitda',
'ev_ebitda',
'ev_receita_líquida',
'ev_fco', 
'ev_fcf', 
'ev_ativo_total',
'preço_vpa',
'preço_capital_giro',
'preço_fcf',
'caixa_equivalentes_caixa'

 ]

dados.drop(columns=colunsa_drop, inplace=True)

In [14]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from typing import Dict, List

from chat_bots import get_secret_key

try:
    api_secret_groq = get_secret_key("GROQ_API_KEY")
except KeyError as exc:
    raise ValueError("API key inválida ou não definida") from exc



In [15]:
from langchain import PromptTemplate

In [16]:
from langchain import PromptTemplate
def chat_bot_fundamentalistas(
    query: str, dados:list,api_secret: str = api_secret_groq
) -> str:
    prompt = PromptTemplate(
      input_variables = ['query', 'dados'],
      template = '''
                                                  You are a fundamental analyst, responsible for assessing the financial health, operational performance, and investment prospects of companies based on financial indicators. Your goal is to produce detailed, well-founded, and actionable analyses for investors.

                                                    The indicators you receive include the following data:

                                                    Resultados financeiros: receita_liquida, resultado_bruto, ebit, ebitda, lucro_liquido, lucro_por_acao.
                                                    Fluxos de caixa: fluxo_caixa_operacional, fluxo_caixa_investimentos, fluxo_caixa_financiamento, fluxo_caixa_livre_tres_meses.
                                                    Investimentos e despesas: capex_tres_meses.
                                                    Valuation: preço_lucro, preço_receita_líquida,  preço_fcf, preço_ebit, ev_ebit, ev_ebitda, enterprise_value, market_cap_empresa.
                                                    Dividendo: dividend_yield.
                                                    Balanço patrimonial: ativo_total, divida_bruta, divida_liquida, patrimonio_liquido, valor_patrimonial_acao.
                                                    Indicadores de rentabilidade e eficiência: retorno_sobre_capital_tangivel_inicial, retorno_sobre_capital_investido_inicial, retorno_sobre_patrimonio_liquido_inicial, margem_bruta, margem_liquida, giro_do_ativo_inicial, alavancagem_financeira.
                                                    Riscos e alavancagem:  passivo_patrimonio_liquido.
                                                    Your Task:
                                                    Analyze indicators: Identify the company’s strengths and weaknesses based on the provided indicators.
                                                    Generate actionable insights: Provide recommendations for investors, such as "buy," "hold," or "sell," with clear justifications based on the data.
                                                    Identify trends: Evaluate historical performance trends and possible implications for the future.
                                                    Communicate results: Present your analysis clearly and concisely, suitable for both beginners and experienced investors.
                                                    Example of Expected Response:

                                                    Evaluated Company: [company name or ticker]
                                                    Analysis Summary: Provide an overall assessment of the company based on the indicators.
                                                    Strengths: Highlight positive metrics, such as high net_income, strong margins, or low net_debt.
                                                    Weaknesses: Identify areas of concern, such as low net_margin or high net_debt_to_ebitda.
                                                    Recommendation: Based on your analysis, provide a recommendation (buy, hold, or sell) with justification.
                                                    
                                                                                                   
        REQUESTED ANALYSIS:
        Question: {query}
        Financial Data: {dados}
        
        
        Always response the questions in Portuguese.

      ''')

    model = ChatGroq(
            api_key=api_secret,
            #model="llama-3.3-70b-versatile",
            model = "llama-3.2-11b-vision-preview",
            temperature=0.5,
            stop_sequences=None,
        )

    llm_chain = prompt | model | StrOutputParser()

    response = llm_chain.invoke(input={'query':query, 'dados': dados})
    
    return response

In [17]:
from langchain_community.document_loaders import DataFrameLoader

In [18]:
dados['datas'] = dados['datas'].astype(str)
loader = DataFrameLoader(dados,page_content_column='datas')

In [19]:
loader = DataFrameLoader(dados,page_content_column='datas')

In [20]:
dados_load = loader.load()

In [21]:
type(dados_load)

list

In [24]:
type(dados_load[0])

langchain_core.documents.base.Document

In [13]:
resposta = chat_bot_fundamentalistas('Analse a empresa petrobras',dados_load)

In [14]:
resposta.split('\n\n')

['**Análise da Empresa Petrobras**',
 '**Resumo da Análise**',
 'A Petrobras é uma empresa de grande porte no setor de petróleo e gás, com uma história de mais de 90 anos de operação. A empresa tem uma presença significativa no mercado brasileiro e internacional, com uma rede de distribuição de petróleo e gás em todo o mundo. A análise abaixo é baseada nos dados financeiros da empresa, que foram coletados a partir de documentos de metadata.',
 '**Pontos Fortes**',
 '* **Receita Líquida**: A receita líquida da Petrobras tem apresentado um crescimento significativo nos últimos anos, com uma média de 10% ao ano.\n* **EBITDA**: O EBITDA da empresa também tem crescido, com uma média de 15% ao ano, o que indica uma boa gestão da empresa.\n* **Margem Bruta**: A margem bruta da Petrobras é alta, com uma média de 50% ao ano, o que indica uma boa capacidade de gerar receita.\n* **Giro do Ativo**: O giro do ativo da empresa é alto, com uma média de 0,5 ao ano, o que indica uma boa capacidade de g

In [92]:


from langchain import PromptTemplate

query = "O que vc pode falar do balanco dessa empresa"

prompt = PromptTemplate(
      input_variables = ['query', 'dados_load'],
      template = '''
                                                  You are a fundamental analyst, responsible for assessing the financial health, operational performance, and investment prospects of companies based on financial indicators. Your goal is to produce detailed, well-founded, and actionable analyses for investors.

                                                    The indicators you receive include the following data:

                                                    Resultados financeiros: receita_liquida, resultado_bruto, ebit, ebitda, lucro_liquido, lucro_por_acao.
                                                    Fluxos de caixa: fluxo_caixa_operacional, fluxo_caixa_investimentos, fluxo_caixa_financiamento, fluxo_caixa_livre_tres_meses.
                                                    Investimentos e despesas: capex_tres_meses.
                                                    Valuation: preço_lucro, preço_receita_líquida,  preço_fcf, preço_ebit, ev_ebit, ev_ebitda, enterprise_value, market_cap_empresa.
                                                    Dividendo: dividend_yield.
                                                    Balanço patrimonial: ativo_total, divida_bruta, divida_liquida, patrimonio_liquido, valor_patrimonial_acao.
                                                    Indicadores de rentabilidade e eficiência: retorno_sobre_capital_tangivel_inicial, retorno_sobre_capital_investido_inicial, retorno_sobre_patrimonio_liquido_inicial, margem_bruta, margem_liquida, giro_do_ativo_inicial, alavancagem_financeira.
                                                    Riscos e alavancagem:  passivo_patrimonio_liquido.
                                                    Your Task:
                                                    Analyze indicators: Identify the company’s strengths and weaknesses based on the provided indicators.
                                                    Generate actionable insights: Provide recommendations for investors, such as "buy," "hold," or "sell," with clear justifications based on the data.
                                                    Identify trends: Evaluate historical performance trends and possible implications for the future.
                                                    Communicate results: Present your analysis clearly and concisely, suitable for both beginners and experienced investors.
                                                    Example of Expected Response:

                                                    Evaluated Company: [company name or ticker]
                                                    Analysis Summary: Provide an overall assessment of the company based on the indicators.
                                                    Strengths: Highlight positive metrics, such as high net_income, strong margins, or low net_debt.
                                                    Weaknesses: Identify areas of concern, such as low net_margin or high net_debt_to_ebitda.
                                                    Recommendation: Based on your analysis, provide a recommendation (buy, hold, or sell) with justification.
                                                    
                                                                                                   
        REQUESTED ANALYSIS:
        Question: {query}
        Financial Data: {dados_load}
        
        
        Always response the questions in Portuguese.

      ''')

model = ChatGroq(
        api_key=api_secret_groq,
        #model="llama-3.3-70b-versatile",
        model = "llama-3.2-11b-vision-preview",
        temperature=0.5,
        stop_sequences=None,
    )

llm_chain = prompt | model | StrOutputParser()

response = llm_chain.invoke(input={'query':query, 'dados_load': dados_load})

In [93]:
response.split('\n\n')

['**Análise da Empresa**',
 'A empresa avaliada é a PETR4, uma empresa de petróleo e gás natural.',
 '**Resumo da Análise**',
 'A PETR4 apresenta um desempenho financeiro sólido, com uma receita líquida crescente ao longo dos anos. A empresa também apresenta uma margem bruta e líquida significativas, o que indica uma eficiência operacional. No entanto, a dívida líquida da empresa é alta em relação ao patrimônio líquido, o que pode ser um sinal de risco.',
 '**Pontos Fortes**',
 '* Receita líquida crescente ao longo dos anos\n* Margem bruta e líquida significativas\n* Eficiência operacional\n* Dívida líquida em relação ao patrimônio líquido é alta, mas está em declínio',
 '**Pontos Fracos**',
 '* Dívida líquida é alta em relação ao patrimônio líquido\n* Fluxo de caixa operacional é variável ao longo dos anos\n* Preço de mercado da empresa é alto em relação ao preço de livro',
 '**Recomendação**',
 'Com base na análise acima, recomendo que os investidores sejam cautelosos ao investir na 